# Simple conditioning plot

In [ ]:
# Import pyMC3 and also arviz for visualisation
import pymc as pm
import arviz as az
import sympy as sp
import pandas as pd
import numpy as np
import statsmodels.api as sm


#plt.rcParams["figure.facecolor"] = "None"
#plt.rcParams["savefig.facecolor"] = "None"
#plt.rcParams['image.cmap'] = 'Accent'
#plt.set_cmap('Accent')


import symseaman as ss
from symseaman.substitute_dynamic_symbols import lambdify, remove_bis_eq, run, standard_substitutes
from symseaman.seaman_symbols import *
from vct.bis_system import BisSystem
from scipy.stats import norm
from vct.regression_ols import product

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('jupyter')
plt.rcParams["figure.figsize"] = (4,1.8)
plt.rcParams["axes.labelsize"] = 12
plt.rcParams["legend.fontsize"] = 12
plt.set_cmap('jet')

In [ ]:
theta_prior = norm(loc=2.5, scale=0.5)
theta_likelihood = norm(loc=3.5, scale=1.0)

In [ ]:
theta_posterior = product(theta_prior, theta_likelihood)

In [ ]:
plt.set_cmap('jet')
rvs = [theta_prior,theta_likelihood,theta_posterior]
xmins = [rv.mean() - 3*rv.std() for rv in rvs]
xmaxs = [rv.mean() + 3*rv.std() for rv in rvs]

xmin = np.min(xmins)
xmax = np.max(xmaxs)

x_ = np.linspace(xmin, 
                 xmax, 300)

fig,ax=plt.subplots()
ax.plot(x_,theta_prior.pdf(x_), label='prior')
ax.plot(x_,theta_likelihood.pdf(x_), label='likelihood')
ax.plot(x_,theta_posterior.pdf(x_), label='posterior')

ax.legend()

ax.set_xlabel(None)
ax.set_yticks([])
ax.set_xticks([])

fig.savefig('theta.svg', transparent=True)

In [ ]:
np.random.seed(42)
dx = x_[1]-x_[0]
theta_priors = theta_prior.pdf(x_)
theta_likelihoods = theta_likelihood.pdf(x_)

In [ ]:
theta_posteriors= (theta_priors*theta_likelihoods)/sum(theta_priors*theta_likelihoods*dx)

In [ ]:

fig,ax=plt.subplots()
ax.plot(x_,theta_priors, color='red', label='prior')
ax.plot(x_,theta_likelihoods, color='blue', label='likelihood')
ax.plot(x_,theta_posteriors, color='purple', label='posterior')

ax.legend()
ax.set_xlabel(r'$\theta$')

In [ ]:
basic_model = pm.Model()

with basic_model:
    # Priors for unknown model parameters
    theta = pm.Normal("theta", mu=theta_prior.mean(), 
                           sigma=theta_prior.std(), 
                     )
    
    sigma = pm.HalfNormal("sigma", sigma=theta_likelihood.std())


    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("fy_obs", mu=theta, sigma=theta_prior.std(), observed=theta_likelihoods)
    #ν = pm.Uniform("ν", 1, 10)
    #Y_obs = pm.StudentT("fy_obs", nu=ν, mu=mu, sigma=sigma, observed=y)

In [ ]:
with basic_model:
    # draw 1000 posterior samples
    trace = pm.sample(draws=1000)

In [ ]:
with basic_model:
    az.plot_posterior(trace,
                  #var_names=['theta','sigma'],
                  textsize=18,
                  point_estimate='mean',
                  rope_color='black')